In [ ]:
import numpy as np
from collections import Counter

class Node:
  def __init__(self, feature=None, threshold=None, left=None, right=None,*,value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value=value

  def _is_leaf_node(self):
    return self.value is not None

class DecisionTree:
  def __init__(self,max_depth=100, min_sample_splits=2, n_features=None, root=None):
    self.max_depth = max_depth
    self.min_sample_splits = min_sample_splits
    self.n_features = n_features
    self.root = root

  def fit(self, X,y):
    self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
    self.root = self._grow_tree(X,y)

  def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _traverse_tree(self,x, node):
    if node._is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x,node.left)
    return self._traverse_tree(x,node.right)

  def _grow_tree(self,X,y,depth=0):
    n_labels = len(np.unique(y))
    n_samples, n_feats = X.shape

    #Check the Stopping criteria.
    if (self.max_depth < depth or n_labels==1 or n_samples < self.min_sample_splits):
      leaf_value = Counter(y).most_common(1)[0][0]
      return Node(value=leaf_value)

    feats_idxs = np.random.choice(n_feats, self.n_features, replace=False)
    #find the best split
    best_feature, best_threshold = self._best_split(X,y,feats_idxs)

    #create the child nodes
    left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth+1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth+1)
    return Node(best_feature, best_threshold, left, right)

  def _best_split(self,X,y,feats_idxs):
    best_gain = -1
    best_feature , best_thr = None, None

    for feats_idx in feats_idxs:
      X_column = X[:, feats_idx]
      thr_idxs = np.unique(X_column)
      for thr in thr_idxs:
        gain = self._information_gain(X_column,y,thr)
        if gain > best_gain:
          best_gain = gain
          best_feature = feats_idx
          best_thr = thr
    return best_feature, best_thr

  def _information_gain(self,X_column, y, thr):
    # Parent entropy
    parent_entropy = self._entropy(y)

    # create child nodes
    left_idxs, right_idxs = self._split(X_column, thr)

    # wt_child_entropy calculation
    n = len(y)
    n_l , n_r = len(y[left_idxs]), len(y[right_idxs])
    e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
    child_entropy = (n_l/n)*e_l + (n_r/n)*e_r

    #calculation information gain
    return parent_entropy - child_entropy

  def _entropy(self,y):
    hist = np.bincount(y)
    ps = hist/len(y)
    return -np.sum([p*np.log(p) for p in ps if p>0])

  def _split(self, X_column,thr):
    left_idxs = np.argwhere(X_column <= thr).flatten()
    right_idxs = np.argwhere(X_column > thr).flatten()
    return left_idxs, right_idxs


In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
data = datasets.load_breast_cancer()
X, y = data.data, data.target

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.2, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(455, 30) (114, 30) (455,) (114,)


In [ ]:
#Model Building
clf = DecisionTree()
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
accuracy_score(y_test, y_pred)

0.9385964912280702